In [7]:
#@title Imports
%reset -f 

In [8]:
import pandas as pd

In [9]:
import pylab
import scipy.io

In [10]:


import numpy as np
from itertools import product as cartesian_prod

import matplotlib.pyplot as plt


from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import os
import argparse
import sys

from sklearn.svm import SVC
np.set_printoptions(precision=2)


def sigmoid(u):
    u = np.maximum(u,-100)
    u = np.minimum(u,100)
    return 1/(1+np.exp(-u))


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import pairwise_distances


In [12]:
class Args:
    def __init__(self):
        self.numlayer=4
        self.numnodes=12
        self.beta=5.
        self.lr=1.
        

In [13]:



args =  Args()

num_layer = args.numlayer
num_neuron = args.numnodes
beta = args.beta
lr=args.lr

saved_epochs = list(range(0,300,1)) + list(range(300,10001,50))
update_value_epochs = list(range(0,10001,100))# 
filename_suffix = str(num_layer)
filename_suffix += "_"+str(num_neuron)
filename_suffix += "_"+str(int(beta))
filename_suffix += "_"+format(lr,".1e")
print(filename_suffix)


no_of_batches=10 #[1,10,100]
weight_decay=0.0
num_hidden_nodes=[num_neuron]*num_layer



device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)


4_12_50_1.0e-03
cpu


**Variable parameters**

In [44]:
#@title Synthetic data
def set_npseed(seed):
	np.random.seed(seed)


def set_torchseed(seed):
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed)
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

#Four mode classification data


def data_gen_decision_tree(num_data=1000, dim=2, seed=0, w_list=None, b_list=None, 
							vals=None, num_levels=2):
	# np.random.seed(6790)
	set_npseed(seed=seed)
	# Construct a complete decision tree with 2**num_levels-1 internal nodes, 
	# e.g. num_levels=2 means there are 3 internal nodes.
	# w_list, b_list is a list of size equal to num_internal_nodes
	# vals is a list of size equal to num_leaf_nodes, with values +1 or -1
	num_internal_nodes = 2**num_levels - 1
	num_leaf_nodes = 2**num_levels
	stats = np.zeros(num_internal_nodes+num_leaf_nodes)

	if vals is None:
		vals = np.arange(0,num_internal_nodes+num_leaf_nodes,1,dtype=np.int32)%2
		vals[:num_internal_nodes] = -99

	if w_list is None:
		w_list = np.random.standard_normal((num_internal_nodes, dim))
		w_list = w_list/np.linalg.norm(w_list, axis=1)[:, None]
		b_list = np.zeros((num_internal_nodes))
	w_list = np.zeros((num_internal_nodes, dim))
	for i in range(num_internal_nodes):
		w_list[i,i]=1.

	data_x = np.random.standard_normal((num_data, dim))
	data_x /= np.sqrt(np.sum(data_x**2, axis=1, keepdims=True))
	relevant_stats = data_x @ w_list.T + b_list
	

	curr_index = np.zeros(shape=(num_data), dtype=int)
	
	for level in range(num_levels):
		nodes_curr_level=list(range(2**level - 1,2**(level+1)-1  ))
		for el in nodes_curr_level:
			relevant_stats[:,el] += b_list[el]
		decision_variable = np.choose(curr_index, relevant_stats.T) 
		# Go down and right if wx+b>0 down and left otherwise. 
		# i.e. 0 -> 1 if w[0]x+b[0]<0 and 0->2 otherwise
		curr_index = (curr_index+1)*2 - (1-(decision_variable > 0))

	bound_dist = np.min(np.abs(relevant_stats), axis=1)
	thres = 0.005
	# thres =0 
	labels = vals[curr_index]
	data_x_pruned = data_x[bound_dist>thres]
	labels_pruned = labels[bound_dist>thres]
	relevant_stats = np.sign(data_x_pruned @ w_list.T + b_list)
	nodes_active = np.zeros((len(data_x_pruned),  num_internal_nodes+num_leaf_nodes), dtype=np.int32)
	for node in range(num_internal_nodes+num_leaf_nodes):
		if node==0:
			stats[node]=len(relevant_stats)
			nodes_active[:,0]=1
			continue
		parent = (node-1)//2
		nodes_active[:,node]=nodes_active[:,parent]
		right_child = node-(parent*2)-1 # 0 means left, 1 means right 1 has children 3,4
		if right_child==1:
			nodes_active[:,node] *= relevant_stats[:,parent]>0
		if right_child==0:
			nodes_active[:,node] *= relevant_stats[:,parent]<0		
		stats = nodes_active.sum(axis=0)
	return ((data_x_pruned, labels_pruned), (w_list, b_list, vals), stats)


# w_list = np.array([[1., 0], [0, 1], [0, 1]])
# b_list = np.array([0, 0.25, -0.25])
# vals = np.array([-99, -99, -99, 0, 1, 1, 0])
num_data = 100000
input_dim=100
seeds = np.random.random_integers(0,10000,100)
seeds=[1387]
# seeds = [2318]
for seed in seeds:
	((data_x, labels), (w_list, b_list, vals), stats) = data_gen_decision_tree(
												dim=input_dim, seed=seed, num_levels=4,
												num_data=num_data)
	seed_set=seed
w_list_old = np.array(w_list)
b_list_old = np.array(b_list)

num_data = len(data_x)
num_train= num_data//2
num_vali = num_data//4
num_test = num_data//4
train_data = data_x[:num_train,:]
train_data_labels = labels[:num_train]

vali_data = data_x[num_train:num_train+num_vali,:]
vali_data_labels = labels[num_train:num_train+num_vali]

test_data = data_x[num_train+num_vali :,:]
test_data_labels = labels[num_train+num_vali :]

/var/folders/pd/w8bnb5rs76g9lp6n7cpjt8ch0000gn/T/ipykernel_52862/2315234387.py:85: DeprecationWarning: This function is deprecated. Please call randint(0, 10000 + 1) instead
  seeds = np.random.random_integers(0,10000,100)


In [45]:
print(train_data.shape)
print(train_data_labels.shape)

print(test_data.shape)
print(test_data_labels.shape)


(27236, 100)
(27236,)
(13618, 100)
(13618,)


In [ ]:

class DLGN_FC(nn.Module):
	def __init__(self, input_dim=None, output_dim=None, num_hidden_nodes=[], beta=30, mode='pwc'):		
		super(DLGN_FC, self).__init__()
		self.num_hidden_layers = len(num_hidden_nodes)
		self.beta=beta  # Soft gating parameter
		self.mode = mode
		self.num_nodes=[input_dim]+num_hidden_nodes+[output_dim]
		self.gating_layers=nn.ModuleList()
		self.value_layers=nn.Parameter(torch.randn([1]+num_hidden_nodes)/100.)
		self.num_layer = len(num_hidden_nodes)
		self.num_hidden_nodes = num_hidden_nodes
		for i in range(self.num_hidden_layers+1):
			if i!=self.num_hidden_layers:
				temp = nn.Linear(self.num_nodes[0], self.num_nodes[i+1], bias=False)
				self.gating_layers.append(temp)

	def set_parameters_with_mask(self, to_copy, parameter_masks):
		# self and to_copy are DLGN_FC objects with same architecture
		# parameter_masks is compatible with dict(to_copy.named_parameters())
		for (name, copy_param) in to_copy.named_parameters():
			copy_param = copy_param.clone().detach()
			orig_param  = self.state_dict()[name]
			if name in parameter_masks:
				param_mask = parameter_masks[name]>0
				orig_param[param_mask] = copy_param[param_mask]
			else:
				orig_param = copy_param.data.detach()
	

								

	def return_gating_functions(self):
		effective_weights = []
		for i in range(self.num_hidden_layers):
			curr_weight = self.gating_layers[i].weight.detach().clone()
			# curr_weight /= torch.norm(curr_weight, dim=1, keepdim=True)
			effective_weights.append(curr_weight)
		return effective_weights
		# effective_weights (and effective biases) is a list of size num_hidden_layers
							

	def forward(self, x):
		

		for el in self.parameters():
			if el.is_cuda:
				device = torch.device('cuda:1')
			else:
				device = torch.device('cpu')
		values=[torch.ones(x.shape).to(device)]
		
		
		for i in range(self.num_hidden_layers):
			fiber = [len(x)]+[1]*self.num_layer
			fiber[i+1] = self.num_hidden_nodes[i]
			fiber = tuple(fiber)
			gate_score = torch.sigmoid( self.beta*(x@self.gating_layers[i].weight.T))#/
			    #   torch.norm(self.gating_layers[i].weight, dim=1, keepdim=True).T) 
			gate_score = gate_score.reshape(fiber) 
			if i==0:
				cp = gate_score
			else:
				cp = cp*gate_score 
		return torch.sum(cp*self.value_layers, dim=(1,2,3,4))

In [ ]:
#@title Train DLGN model
def train_dlgn (DLGN_obj, train_data_curr,vali_data_curr,test_data_curr,
				train_labels_curr,test_labels_curr,vali_labels_curr,
				parameter_mask=dict()):
	# DLGN_obj is the initial network
	# parameter_mask is a dictionary compatible with dict(DLGN_obj.named_parameters())
	# if a key corresponding to a named_parameter is not present it is assumed to be all ones (i.e it will be updated)
	
	# Assuming that we are on a CUDA machine, this should print a CUDA device:
	
	# Speed up of a factor of over 40 by using GPU instead of CPU
	# Final train loss of 0.02 and test acc of 74%
	device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
	# device = torch.device('cpu')
	DLGN_obj.to(device)

	criterion = nn.CrossEntropyLoss()




	optimizer = optim.SGD(DLGN_obj.parameters(), lr=lr)



	train_data_torch = torch.Tensor(train_data_curr)
	vali_data_torch = torch.Tensor(vali_data_curr)
	test_data_torch = torch.Tensor(test_data_curr)

	train_labels_torch = torch.tensor(train_labels_curr, dtype=torch.int64)
	test_labels_torch = torch.tensor(test_labels_curr, dtype=torch.int64)
	vali_labels_torch = torch.tensor(vali_labels_curr, dtype=torch.int64)

	num_batches = no_of_batches
	batch_size = len(train_data_curr)//num_batches
	losses=[]
	DLGN_obj_store = []
	best_vali_error = len(vali_labels_curr)
	

	# print("H3")
	# print(DLGN_params)
	debug_models= []
	train_losses = []
	tepoch = tqdm(range(saved_epochs[-1]+1))
	for epoch in tepoch:  # loop over the dataset multiple times
		if epoch in update_value_epochs:
			# updating the value pathdim vector by optimising 

			train_preds =DLGN_obj(torch.Tensor(train_data_curr).to(device)).reshape((-1,1))
			criterion = nn.CrossEntropyLoss()
			outputs = torch.cat((-1*train_preds,train_preds), dim=1)
			targets = torch.tensor(train_labels_curr, dtype=torch.int64).to(device)
			
			train_loss = criterion(outputs, targets)
			print("Loss lefore updating value_net at epoch", epoch, " is ", train_loss)
			print("Total path abs value", torch.abs(DLGN_obj.value_layers.cpu().detach()).sum().numpy())

			ew = DLGN_obj.return_gating_functions()
			cp_feat1 = sigmoid(beta*np.dot(train_data_curr,ew[0].cpu().T).reshape(-1,num_neuron,1,1,1))
			cp_feat2 = sigmoid(beta*np.dot(train_data_curr,ew[1].cpu().T).reshape(-1,1,num_neuron,1,1))
			cp_feat3 = sigmoid(beta*np.dot(train_data_curr,ew[2].cpu().T).reshape(-1,1,1,num_neuron,1))
			cp_feat4 = sigmoid(beta*np.dot(train_data_curr,ew[3].cpu().T).reshape(-1,1,1,1,num_neuron))
			cp_feat = cp_feat1 * cp_feat2 * cp_feat3 * cp_feat4
			cp_feat_vec = cp_feat.reshape((len(cp_feat),-1))

			clf = LogisticRegression(C=0.03, fit_intercept=False,max_iter=1000, penalty="l1", solver='liblinear')
			clf.fit(2*cp_feat_vec, train_labels_curr)
			value_wts  = clf.decision_function(np.eye(num_neuron**num_layer)).reshape(1,num_neuron,num_neuron,num_neuron, num_neuron)
			
			A= DLGN_obj.value_layers.detach()
			A[:] = torch.Tensor(value_wts)

			train_preds =DLGN_obj(torch.Tensor(train_data_curr).to(device)).reshape((-1,1))
			criterion = nn.CrossEntropyLoss()
			outputs = torch.cat((-1*train_preds,train_preds), dim=1)
			targets = torch.tensor(train_labels_curr, dtype=torch.int64).to(device)
			train_loss = criterion(outputs, targets)
			print("Loss after updating value_net at epoch", epoch, " is ", train_loss)			
			print("Total path abs value", torch.abs(DLGN_obj.value_layers.cpu().detach()).sum().numpy())
		if epoch in saved_epochs:
			DLGN_obj_copy = deepcopy(DLGN_obj)
			DLGN_obj_copy.to(torch.device('cpu'))
			DLGN_obj_store.append(DLGN_obj_copy)
		
		for batch_start in range(0,len(train_data_curr),batch_size):
			if (batch_start+batch_size)>len(train_data_curr):
				break
			optimizer.zero_grad()
			inputs = train_data_torch[batch_start:batch_start+batch_size]
			targets = train_labels_torch[batch_start:batch_start+batch_size].reshape(batch_size)
			criterion = nn.CrossEntropyLoss()
			inputs = inputs.to(device)
			targets = targets.to(device)
			preds = DLGN_obj(inputs).reshape(-1,1)
			# preds_clone = preds.detach().clone().cpu().numpy()[:,0]
			# targets_clone = targets.detach().clone().cpu().numpy()
			# coeff = (0.5-targets_clone)/(sigmoid(2*preds_clone)-targets_clone)
			# print(coeff.shape)
			
			# print(coeff.min())
			# print(coeff.mean())
			# print(coeff.max())
			outputs = torch.cat((-1*preds, preds), dim=1)
			loss = criterion(outputs, targets)
			# loss = loss*torch.tensor(coeff, device=device)	
			# loss = loss.mean()		
			loss.backward()
			for name,param in DLGN_obj.named_parameters():
				if "val" in name:
					param.grad *= 0.0
				if "gat" in name:
					param.grad *= 1.0
			optimizer.step()

		train_preds =DLGN_obj(torch.Tensor(train_data_curr).to(device)).reshape(-1,1)
		criterion = nn.CrossEntropyLoss()
		outputs = torch.cat((-1*train_preds,train_preds), dim=1)
		targets = torch.tensor(train_labels_curr, dtype=torch.int64).to(device)
		train_loss = criterion(outputs, targets)
		if epoch%5 == 0:
			print("Loss after updating at epoch ", epoch, " is ", train_loss)
			test_preds =DLGN_obj(test_data_torch.to(device)).reshape(-1,1)
			test_preds = test_preds.detach().cpu().numpy()
			print("Test error=",np.sum(test_labels_curr != (np.sign(test_preds[:,0])+1)//2 ))
		if train_loss < 0.005:
			break
		if np.isnan(train_loss.detach().cpu().numpy()):
			break

		losses.append(train_loss.cpu().detach().clone().numpy())
		inputs = vali_data_torch.to(device)
		targets = vali_labels_torch.to(device)
		preds =DLGN_obj(inputs).reshape(-1,1)
		vali_preds = torch.cat((-1*preds, preds), dim=1)
		vali_preds = torch.argmax(vali_preds, dim=1)
		vali_error= torch.sum(targets!=vali_preds)
		if vali_error < best_vali_error:
			DLGN_obj_return = deepcopy(DLGN_obj)
			best_vali_error = vali_error
	plt.figure()
	plt.title("DLGN loss vs epoch")
	plt.plot(losses)
	if not os.path.exists('figures'):
		os.mkdir('figures')

	filename = 'figures/'+filename_suffix +'.pdf'
	plt.savefig(filename)
	DLGN_obj_return.to(torch.device('cpu'))
	device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
	# device = torch.device('cpu')
	return train_losses, DLGN_obj_return, DLGN_obj_store, losses, debug_models

**Training a DLGN model**

In [ ]:
set_torchseed(41972)
# set_torchseed(5612)
DLGN_init= DLGN_FC(input_dim=input_dim, output_dim=1, num_hidden_nodes=num_hidden_nodes, beta=beta)



In [ ]:
for name,parameter in DLGN_init.named_parameters():
    print(name)
    print(parameter.shape)

In [ ]:
train_parameter_masks=dict()
for name,parameter in DLGN_init.named_parameters():
    if "val" in name:
        train_parameter_masks[name]=torch.ones_like(parameter)# Updating all value network layers
    if "gat" in name:
        train_parameter_masks[name]=torch.ones_like(parameter)
    train_parameter_masks[name].to(device)


        






set_torchseed(5000)
train_losses, DLGN_obj_final, DLGN_obj_store, losses , debug_models= train_dlgn(train_data_curr=train_data,
                                            vali_data_curr=vali_data,
                                            test_data_curr=test_data,
                                            train_labels_curr=train_data_labels,
                                            vali_labels_curr=vali_data_labels,
                                            test_labels_curr=test_data_labels,
                                            DLGN_obj=deepcopy(DLGN_init),
                                            parameter_mask=train_parameter_masks,
                                            )


torch.cuda.empty_cache() 
losses=np.array(losses)

print(DLGN_obj_store[-1].beta)

In [ ]:
if not os.path.exists('outputs'):
    os.mkdir('outputs')
device=torch.device('cpu')
train_preds =DLGN_obj_final(torch.Tensor(train_data).to(device)).reshape(-1,1)
criterion = nn.CrossEntropyLoss()
outputs = torch.cat((-1*train_preds,train_preds), dim=1)
targets = torch.tensor(train_data_labels, dtype=torch.int64)
train_loss = criterion(outputs, targets)
train_preds = train_preds.detach().numpy()
filename = 'outputs/'+filename_suffix+'.txt'
original_stdout = sys.stdout


In [ ]:
with open(filename,'w') as f:
    sys.stdout = f
    print("Setup:")
    print("Num neurons : ", DLGN_obj_final.num_nodes)
    print(" Beta :", DLGN_obj_final.beta)
    print(" lr :", lr)
    print("=======================")
    print(train_losses)
    print("==========Best validated model=============")
    print("Train error=",np.sum(train_data_labels != (np.sign(train_preds[:,0])+1)//2 ))
    print("Train loss = ", train_loss)
    print("Num_train_data=",len(train_data_labels))
    sys.stdout = original_stdout


In [ ]:
test_preds =DLGN_obj_final(torch.Tensor(test_data)).reshape(-1,1)
test_preds = test_preds.detach().numpy()
filename = 'outputs/'+filename_suffix+'.txt'
original_stdout = sys.stdout
with open(filename,'a') as f:
    sys.stdout = f
    print("Test error=",np.sum(test_data_labels != (np.sign(test_preds[:,0])+1)//2 ))
    print("Num_test_data=",len(test_data_labels))
    sys.stdout = original_stdout


In [ ]:
w_list = np.concatenate((w_list_old,-w_list_old),axis=0)

effective_weights = DLGN_obj_store[0].return_gating_functions()
wts_list_init=[]
for layer in range(0,len(effective_weights)):
    wts =  np.array(effective_weights[layer].detach().clone().numpy())+0.001
    wts /= np.linalg.norm(wts, axis=1)[:,None]
    wts_list_init.append(wts)
wts_list_init = np.concatenate(wts_list_init)


effective_weights = DLGN_obj_final.return_gating_functions()
if np.isnan(train_loss.detach().numpy()):
    filename = 'outputs/'+filename_suffix+'.txt'
    original_stdout = sys.stdout
    with open(filename,'a') as f:
        sys.stdout = f
        print("Nan error")
        sys.stdout = original_stdout
else:
    wts_list=[]
    for layer in range(len(effective_weights)):
        wts =  np.array(effective_weights[layer].detach().clone().numpy()) + 0.001
        wts /= np.linalg.norm(wts, axis=1)[:,None]
        wts_list.append(wts)
    wts_list = np.concatenate(wts_list)

    pd0 =  pairwise_distances(w_list,wts_list_init)
    pd1 =  pairwise_distances(w_list,wts_list)


    filename = 'outputs/'+filename_suffix+'.txt'
    original_stdout = sys.stdout
    with open(filename,'a') as f:
        sys.stdout = f
        print("Shape of decision tree node hyperplanes ", w_list.shape)
        print("Shape of all halfspace directions of DLGN", wts_list.shape)
        print("Distance of closest init DLGN halfspace to each labelling func hyperplane \n", pd0.min(axis=1)[:len(w_list_old)])
        print(pd0.min(axis=1)[len(w_list_old):])
        print("Distance of closest lrnd DLGN halfspace to each labelling func hyperplane \n", pd1.min(axis=1)[:len(w_list_old)])
        print(pd1.min(axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.8 of the Dtree hyperplanes \n", np.sum(pd1<0.8, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.8, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.6 of the Dtree hyperplanes \n", np.sum(pd1<0.6, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.6, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.4 of the Dtree hyperplanes \n", np.sum(pd1<0.4, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.4, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.3 of the Dtree hyperplanes \n", np.sum(pd1<0.3, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.3, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.2 of the Dtree hyperplanes \n", np.sum(pd1<0.2, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.2, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.1 of the Dtree hyperplanes \n", np.sum(pd1<0.1, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.1, axis=1)[len(w_list_old):])
        print("=========================================")
        sys.stdout = original_stdout




In [ ]:
for epoch_index in tqdm(range(len(DLGN_obj_store))):
    train_preds =DLGN_obj_store[epoch_index](torch.Tensor(train_data).to(device)).reshape(-1,1)
    criterion = nn.CrossEntropyLoss()
    outputs = torch.cat((-1*train_preds,train_preds), dim=1)
    targets = torch.tensor(train_data_labels, dtype=torch.int64)
    train_loss = criterion(outputs, targets)


    if np.isnan(train_loss.detach().numpy()):
        filename = 'outputs/'+filename_suffix+'.txt'
        original_stdout = sys.stdout
        with open(filename,'a') as f:
            sys.stdout = f
            print("Nan error")
            sys.stdout = original_stdout
            continue

    effective_weights = DLGN_obj_store[epoch_index].return_gating_functions()
    wts_list=[]
    wts_list_unnorm=[]
    for layer in range(len(effective_weights)):
        wts =  np.array(effective_weights[layer].detach().clone().numpy())+0.001
        wts_list_unnorm.append(np.array(wts))
        wts /= np.linalg.norm(wts, axis=1)[:,None]
        wts_list.append(wts)
    wts_list = np.concatenate(wts_list)
    wts_list_unnorm = np.concatenate(wts_list_unnorm)
    pd0 =  pairwise_distances(w_list,wts_list_init)
    pd1 =  pairwise_distances(w_list,wts_list)
    pd_w_list = pairwise_distances(w_list, w_list)
    w_list_random = np.random.standard_normal(w_list.shape)
    w_list_random /= np.linalg.norm(w_list_random, axis=1)[:,None]

    pd4 = pairwise_distances(w_list_random,wts_list)


    filename = 'outputs/'+filename_suffix+'.txt'
    original_stdout = sys.stdout
    with open(filename,'a') as f:
        sys.stdout = f
        print("===================================")
        print(epoch_index)
        print(saved_epochs[epoch_index])
        print("===================================")
        train_preds = train_preds.detach().numpy()
        test_preds =DLGN_obj_store[epoch_index](torch.Tensor(test_data)).reshape(-1,1)
        test_preds = test_preds.detach().numpy()
        print("Train error=",np.sum(train_data_labels != (np.sign(train_preds[:,0])+1)//2 ))
        print("Num_train_data=",len(train_data_labels))
        print("Train loss=",train_loss.detach())
        print("Test error=",np.sum(test_data_labels != (np.sign(test_preds[:,0])+1)//2 ))
        print("Num_test_data=",len(test_data_labels))
        print("===================================")

        print("Shape of decision tree node hyperplanes ", w_list.shape)
        print("Shape of all halfspace directions of DLGN", wts_list.shape)
        print("Distance of closest init DLGN halfspace to each labelling func hyperplane \n", pd0.min(axis=1)[:len(w_list_old)])
        print(pd0.min(axis=1)[len(w_list_old):])
        print("Distance of closest lrnd DLGN halfspace to each labelling func hyperplane \n", pd1.min(axis=1)[:len(w_list_old)])
        print(pd1.min(axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.8 of the Dtree hyperplanes \n", np.sum(pd1<0.8, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.8, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.6 of the Dtree hyperplanes \n", np.sum(pd1<0.6, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.6, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.4 of the Dtree hyperplanes \n", np.sum(pd1<0.4, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.4, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.3 of the Dtree hyperplanes \n", np.sum(pd1<0.3, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.3, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.2 of the Dtree hyperplanes \n", np.sum(pd1<0.2, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.2, axis=1)[len(w_list_old):])
        print("Number of halfspaces within distance 0.1 of the Dtree hyperplanes \n", np.sum(pd1<0.1, axis=1)[:len(w_list_old)])
        print(np.sum(pd1<0.1, axis=1)[len(w_list_old):])
        print("=========================================")
        print("============Norm, max, argmax of hidden neurons, index, sign at argmax===========")
        a = np.linalg.norm(wts_list_unnorm, axis=1)
        b = np.max(np.abs(wts_list_unnorm), axis=1)
        c = np.argmax(np.abs(wts_list_unnorm), axis=1) 
        d = np.argsort(-a)
        for i in range(len(a)):
            print(a[d[i]], b[d[i]], c[d[i]], d[i],np.sign(wts_list_unnorm[d[i],c[d[i]]]) )
        print("=========================================")
        sys.stdout = original_stdout    


In [ ]:
ew_store = np.zeros((len(DLGN_obj_store),num_layer,num_neuron,input_dim))
val_store = np.zeros( [len(DLGN_obj_store)]+ [num_neuron]*num_layer )
for i,model in enumerate(DLGN_obj_store):
    ew = DLGN_obj_store[i].return_gating_functions()
    ew_store[i,0] = ew[0].numpy()
    ew_store[i,1] = ew[1].numpy()
    ew_store[i,2] = ew[2].numpy()
    ew_store[i,3] = ew[3].numpy()
    val_store[i] = model.value_layers.detach().clone().numpy()
# np.savez("npzfiles/DLGN_outer_100dim_microanalysis_init_zero_thres.npz", ew_store, val_store)

# print(ew_store.shape)
# print(val_store.shape)

In [ ]:
print(chumma)

In [ ]:
arrays = np.load("npzfiles/DLGN_outer_100dim_microanalysis_init.npz")
# arrays = np.load("npzfiles/DLGN_outer_100dim_microanalysis_init_zero_thres.npz")
ew_store_outer = arrays["arr_0"]
val_store_outer = arrays["arr_1"]
print(ew_store_outer.shape)
print(val_store_outer.shape)


In [ ]:
temp = np.sum(np.abs(val_store_outer), axis=0)
print(np.sum(temp>0))

In [ ]:
train_loss_curve = []
test_loss_curve = []
test_error_curve = []
train_loss_vecs = []
for epoch_index in range(len(saved_epochs)):
    DLGN_obj = DLGN_FC(input_dim=input_dim, output_dim=1, num_hidden_nodes=num_hidden_nodes, beta=beta)
    temp = deepcopy(val_store_outer[epoch_index])
    # temp[np.abs(temp)<0.1]=0
    DLGN_obj.value_layers.data = torch.Tensor(temp)
    DLGN_obj.gating_layers[0].weight.data = torch.Tensor(deepcopy(ew_store_outer[epoch_index,0]))
    DLGN_obj.gating_layers[1].weight.data = torch.Tensor(deepcopy(ew_store_outer[epoch_index,1]))
    DLGN_obj.gating_layers[2].weight.data = torch.Tensor(deepcopy(ew_store_outer[epoch_index,2]))
    DLGN_obj.gating_layers[3].weight.data = torch.Tensor(deepcopy(ew_store_outer[epoch_index,3]))
    device = torch.device("cpu")
    DLGN_obj.to(device)

    train_preds = DLGN_obj(torch.Tensor(train_data).to(device)).detach().clone().numpy()
    train_loss_vec = -np.log(sigmoid(train_preds*2*(2*train_data_labels - 1)))
    test_preds = DLGN_obj(torch.Tensor(test_data).to(device)).detach().clone().numpy()
    test_loss_vec = -np.log(sigmoid(test_preds*2*(2*test_data_labels - 1)))
    train_loss_vecs.append(train_loss_vec)
    train_loss_curve.append(train_loss_vec.mean())
    test_loss_curve.append(test_loss_vec.mean())
    test_error_curve.append(np.sum(test_loss_vec>np.log(2))) 




In [ ]:
plt.plot(saved_epochs, train_loss_curve)
plt.plot(saved_epochs, test_loss_curve)

plt.figure()
plt.plot(saved_epochs,test_error_curve)


In [ ]:
a= np.linalg.norm(ew_store_outer, axis=3)
b= np.max(np.abs(ew_store_outer), axis=3)
c= np.argmax(np.abs(ew_store_outer), axis=3)
epoch_index = -1
ew_store_sign = np.sign(ew_store_outer)
large_val_paths = np.where(np.abs(val_store_outer[epoch_index])>0.01)



In [ ]:
for l in [1]:
    for k in range(num_neuron):
        plt.figure()
        plt.title("l="+str(l)+" k="+str(k)+" argmax = "+str(c[-1,l,k]))
        for i in range(15):
            plt.plot(saved_epochs, ew_store_outer[:,l,k,i])

In [ ]:
select013715   = (train_data[:,0]<0) * (train_data[:,1]<0) * (train_data[:,3]<0) *  (train_data[:,7]<0) #
select013716   = (train_data[:,0]<0) * (train_data[:,1]<0) * (train_data[:,3]<0) *  (train_data[:,7]>0) #
select013817   = (train_data[:,0]<0) * (train_data[:,1]<0) * (train_data[:,3]>0) *  (train_data[:,8]<0) #
select013818   = (train_data[:,0]<0) * (train_data[:,1]<0) * (train_data[:,3]>0) *  (train_data[:,8]>0) #
select014919   = (train_data[:,0]<0) * (train_data[:,1]>0) * (train_data[:,4]<0) *  (train_data[:,9]<0) #
select0149120  = (train_data[:,0]<0) * (train_data[:,1]>0) * (train_data[:,4]<0) *  (train_data[:,9]>0) #
select01491021 = (train_data[:,0]<0) * (train_data[:,1]>0) * (train_data[:,4]>0) * (train_data[:,10]<0) #
select01491022 = (train_data[:,0]<0) * (train_data[:,1]>0) * (train_data[:,4]>0) * (train_data[:,10]>0) #

select0251123   = (train_data[:,0]>0) * (train_data[:,2]<0) * (train_data[:,5]<0) * (train_data[:,11]<0) #
select0251124   = (train_data[:,0]>0) * (train_data[:,2]<0) * (train_data[:,5]<0) * (train_data[:,11]>0) #
select0251225   = (train_data[:,0]>0) * (train_data[:,2]<0) * (train_data[:,5]>0) * (train_data[:,12]<0) #
select0251226   = (train_data[:,0]>0) * (train_data[:,2]<0) * (train_data[:,5]>0) * (train_data[:,12]>0) #
select0261327   = (train_data[:,0]>0) * (train_data[:,2]>0) * (train_data[:,6]<0) * (train_data[:,13]<0) #
select0261328   = (train_data[:,0]>0) * (train_data[:,2]>0) * (train_data[:,6]<0) * (train_data[:,13]>0) #
select0261429   = (train_data[:,0]>0) * (train_data[:,2]>0) * (train_data[:,6]>0) * (train_data[:,14]<0) #
select0261430   = (train_data[:,0]>0) * (train_data[:,2]>0) * (train_data[:,6]>0) * (train_data[:,14]>0) #

select_subsets_train_leaves = [select013715, select013716, select013817, select013818, select014919, select0149120, select01491021, select01491022,
                               select0251123, select0251124,  select0251225, select0251226, select0261327, select0261328, select0261429, select0261430 ]

In [ ]:
l,n = 0,0

plt.plot(saved_epochs[:310], np.abs(ew_store_outer[:310,l,n,0]), "-");
plt.plot(saved_epochs[:310], np.abs(ew_store_outer[:310,l,n,1:15]), "-");
plt.plot(saved_epochs[:310], np.abs(ew_store_outer[:310,l,n,15:100]), "--");

In [ ]:
epoch_index=-1
mult=1.
ew =  ew_store_outer[epoch_index]
cp_feat1 = sigmoid(mult*beta*np.dot(train_data,ew[0].T).reshape(-1,num_neuron,1,1,1))
cp_feat2 = sigmoid(mult*beta*np.dot(train_data,ew[1].T).reshape(-1,1,num_neuron,1,1))
cp_feat3 = sigmoid(mult*beta*np.dot(train_data,ew[2].T).reshape(-1,1,1,num_neuron,1))
cp_feat4 = sigmoid(mult*beta*np.dot(train_data,ew[3].T).reshape(-1,1,1,1,num_neuron))
cp_feat = cp_feat1 * cp_feat2 * cp_feat3 * cp_feat4
cp_feat_vec = cp_feat.reshape((len(cp_feat),-1))
cp_feat_vec_select = cp_feat_vec[:,np.abs(val_store_outer[epoch_index].reshape(-1))>0.00000001]
large_val_paths = np.where(np.abs(val_store_outer[epoch_index])>0.00000001)

large_vals = np.zeros(len(large_val_paths[0]))
for i in range(len(large_val_paths[0])):
    i1 = large_val_paths[0][i]
    i2 = large_val_paths[1][i]
    i3 = large_val_paths[2][i]
    i4 = large_val_paths[3][i]
    large_vals[i] = val_store_outer[epoch_index,i1,i2,i3,i4]
    print("============================================")
    print(c[-1,0,i1], c[-1,1,i2], c[-1,2,i3], c[-1,3,i4])
    print(b[-1,0,i1], b[-1,1,i2], b[-1,2,i3], b[-1,3,i4])
    print(ew_store_outer[-1,0,i1,c[-1,0,i1]], ew_store_outer[-1,1,i2,c[-1,1,i2]],
           ew_store_outer[-1,2,i3,c[-1,2,i3]], ew_store_outer[-1,3,i4,c[-1,3,i4]])
    print(a[-1,0,i1], a[-1,1,i2], a[-1,2,i3], a[-1,3,i4])


In [ ]:
path_statistics= list()
for l in range(4):
    for i1 in [0]:
        for i2 in range(12):
            for i3 in range(12):
                for i4 in range(12):
                    if np.abs(val_store_outer[epoch_index,i1,i2,i3,i4])<0.000000001:
                        continue
                
                    select1 = cp_feat1[:,i1,0,0,0]>0.5
                    select2 = cp_feat2[:,0,i2,0,0]>0.5
                    select3 = cp_feat3[:,0,0,i3,0]>0.5
                    select4 = cp_feat4[:,0,0,0,i4]>0.5

                    if l==0:
                        selects = [select2, select3, select4, (cp_feat1[:,i1,0,0,0]>0.1)&(cp_feat1[:,i1,0,0,0]<0.9), select1 ]                    
                    if l==1:
                        selects = [select1, select3, select4, (cp_feat2[:,0,i2,0,0]>0.1)&(cp_feat2[:,0,i2,0,0]<0.9), select2]
                    if l==2:
                        selects = [select1, select2, select4, (cp_feat3[:,0,0,i3,0]>0.1)&(cp_feat3[:,0,0,i3,0]<0.9), select3]
                    if l==3:
                        selects = [select1, select2, select3, (cp_feat4[:,0,0,0,i4]>0.1)&(cp_feat4[:,0,0,0,i4]<0.9), select4]

                    entry =dict()
                    entry['l'] = l
                    entry['i1'] = i1
                    entry['i2'] = i2
                    entry['i3'] = i3
                    entry['i4'] = i4
                    entry['val'] = val_store_outer[epoch_index,i1,i2,i3,i4]
                    entry['num_data_thru'] = np.sum(selects[0]*selects[1]*selects[2]*selects[3] )
                    entry["argmax layer 1"] = c[epoch_index,0,i1]
                    entry["argmax layer 2"] = c[epoch_index,1,i2]
                    entry["argmax layer 3"] = c[epoch_index,2,i3]
                    entry["argmax layer 4"] = c[epoch_index,3,i4]
                    entry["max layer1"] = ew_store_outer[epoch_index,0,i1,c[epoch_index,0,i1]]
                    entry["max layer2"] = ew_store_outer[epoch_index,1,i2,c[epoch_index,1,i2]]
                    entry["max layer3"] = ew_store_outer[epoch_index,2,i3,c[epoch_index,2,i3]]
                    entry["max layer4"] = ew_store_outer[epoch_index,3,i4,c[epoch_index,3,i4]]

                    selected_data = train_data[selects[0]*selects[1]*selects[2]*selects[3]]
                    selected_data_labels = train_data_labels[selects[0]*selects[1]*selects[2]*selects[3]]
                    selected_data_cp_feat_vec_select = cp_feat_vec_select[selects[0]*selects[1]*selects[2]*selects[3]]
                    selected_data_preds = sigmoid(2*np.dot(selected_data_cp_feat_vec_select,large_vals))
                    selected_data_grad_imp = (selected_data_preds - selected_data_labels)/(0.5 - selected_data_labels)

                    preds = sigmoid(2*np.dot(cp_feat_vec_select,large_vals))
                    grad_imp = (preds - train_data_labels)/(0.5 - train_data_labels)

                    entry["Conf_mats_ODT_node_classifiers"] = np.zeros((16,2,2))

                    for i in range(16):
                        active_sensitive_indices_right = selects[0]*selects[1]*selects[2]*selects[3]* (train_data[:,i]>0)
                        active_sensitive_indices_left = selects[0]*selects[1]*selects[2]*selects[3]* (train_data[:,i]<0)
                        num_pos_right = np.sum(train_data_labels[active_sensitive_indices_right]* grad_imp[active_sensitive_indices_right])
                        num_neg_right = np.sum( (1-train_data_labels[active_sensitive_indices_right]) * grad_imp[active_sensitive_indices_right] )
                        num_pos_left = np.sum(train_data_labels[active_sensitive_indices_left] * grad_imp[active_sensitive_indices_left])
                        num_neg_left = np.sum( (1-train_data_labels[active_sensitive_indices_left]) *  grad_imp[active_sensitive_indices_left])
                        entry["Conf_mats_ODT_node_classifiers"][i,0,0] = num_neg_left
                        entry["Conf_mats_ODT_node_classifiers"][i,0,1] = num_neg_right
                        entry["Conf_mats_ODT_node_classifiers"][i,1,0] = num_pos_left
                        entry["Conf_mats_ODT_node_classifiers"][i,1,1] = num_pos_right

                    entry["Sum improvement"] = np.zeros((2,16))
                    for i in range(16):
                        entry["Sum improvement"][0,i] = entry["val"]* (entry["Conf_mats_ODT_node_classifiers"][i,1,0] - entry["Conf_mats_ODT_node_classifiers"][i,0,0])
                        entry["Sum improvement"][1,i] = entry["val"]* (entry["Conf_mats_ODT_node_classifiers"][i,1,1] - entry["Conf_mats_ODT_node_classifiers"][i,0,1])


                    path_statistics.append(entry)
                    
selected_path_stats = [el for el in path_statistics if el["l"]==0 and el["i1"]==0]


In [ ]:
layer=0
neuron_index = 0


In [ ]:
# np.set_printoptions(precision=3)
sum_improvement_total = np.zeros((2,16))
print(len(selected_path_stats))
for j in range(len(selected_path_stats)):
    path_stat = selected_path_stats[j]
    print("Neuron index", path_stat["i1"], path_stat["i2"], path_stat["i3"], path_stat["i4"])
    print("Argmax of gating HS", path_stat["argmax layer 1"], path_stat["argmax layer 2"], path_stat["argmax layer 3"], path_stat["argmax layer 4"])
    print("Max of gating HS",path_stat["max layer1"], path_stat["max layer2"], path_stat["max layer3"], path_stat["max layer4"])
    print("Path value",  path_stat["val"])
    print("confmat 0", "\n", path_stat["Conf_mats_ODT_node_classifiers"][0])
    print("confmat 1", "\n", path_stat["Conf_mats_ODT_node_classifiers"][1])
    print("confmat 15", "\n", path_stat["Conf_mats_ODT_node_classifiers"][15])
    print(" Sum Improvement", "\n", path_stat["Sum improvement"] )
    print("==========")
    sum_improvement_total += path_stat["Sum improvement"]
grads = sum_improvement_total[1,:]-sum_improvement_total[0,:]
print("Total improvement: \n", grads )
curr = ew_store_outer[epoch_index,layer, neuron_index, :16]
print("Current HS: \n", curr)
print("Normalised tot improv.: \n", grads/np.linalg.norm(grads))

In [ ]:
epoch_index=99
mult=1.
ew =  ew_store_outer[epoch_index]
cp_feat1 = sigmoid(mult*beta*np.dot(train_data,ew[0].T).reshape(-1,num_neuron,1,1,1))
cp_feat2 = sigmoid(mult*beta*np.dot(train_data,ew[1].T).reshape(-1,1,num_neuron,1,1))
cp_feat3 = sigmoid(mult*beta*np.dot(train_data,ew[2].T).reshape(-1,1,1,num_neuron,1))
cp_feat4 = sigmoid(mult*beta*np.dot(train_data,ew[3].T).reshape(-1,1,1,1,num_neuron))
cp_feat = cp_feat1 * cp_feat2 * cp_feat3 * cp_feat4
cp_feat_vec = cp_feat.reshape((len(cp_feat),-1))
cp_feat_vec_select = cp_feat_vec[:,np.abs(val_store_outer[epoch_index].reshape(-1))>0.00000001]
large_val_paths = np.where(np.abs(val_store_outer[epoch_index])>0.00000001)
large_vals = val_store_outer[epoch_index][large_val_paths]

In [ ]:
tau = 0.00
cp_feat_vec_select_thresholded = np.zeros(cp_feat_vec_select.shape)
cp_feat_vec_select_thresholded[cp_feat_vec_select>tau]  = cp_feat_vec_select[cp_feat_vec_select>tau]
train_preds = np.dot(cp_feat_vec_select_thresholded, large_vals)
plt.title("Mult ="+str(mult)+" Epoch index = "+str(epoch_index))
plt.hist(train_preds[train_data_labels==0], bins=50);
plt.hist(train_preds[train_data_labels==1], bins=50);

In [ ]:
print( ((sigmoid(2*train_preds[train_data_labels==1]) - 1.)/(0.5-1)) .mean())
print( ((sigmoid(2*train_preds[(train_data_labels==1) & (np.abs(train_data[:,0])<0.03)]) - 1)/(0.5-1)).mean())

In [ ]:

plt.figure()
plt.hist(train_preds[train_data_labels==1], bins=50);
plt.figure()
plt.hist(train_preds[(train_data_labels==1) & (np.abs(train_data[:,0]<0.04))], bins=50);